# 📚 AkademskaKniga.mk - Book Dataset Cleaning & Preprocessing

In [3]:
import pandas as pd
import numpy as np

In [4]:
df= pd.read_csv('akademska_books.csv', encoding='utf-8-sig') 

In [5]:
df.head()

,Title,Author,Real Price,Sale Price,Sale,Category,Retrieved At
0,Accounting,"Warren, Carl S.",6199.0,4959 Мкд.,1,Accounting & Finance,2025-05-15
1,Pricing in General I,"Parodi, Pietro",5399.0,4319 Мкд.,1,Accounting & Finance,2025-05-15
2,Behavioral Finance i,"Kubinska, Elzbieta",9399.0,7519 Мкд.,1,Accounting & Finance,2025-05-15
3,Handbook of Alternat,-,10099.0,8079 Мкд.,1,Accounting & Finance,2025-05-15
4,Using QuickBooks (R),"Barbara), Glenn Owen",4999.0,3999 Мкд.,1,Accounting & Finance,2025-05-15


TODO:
- -in author
- check for ?
check for NA,NA,NA... rows
- extract mkd
- convert sale to bool
- convert retrieved_at to datetime
- check for duplicates
- check for multiple authors (, and)

In [6]:
df.isnull().sum()

Title           629
Author          631
Real Price      629
Sale Price      629
Sale              0
Category          0
Retrieved At      0
dtype: int64

In [7]:
df.shape[0]

25165